In [1]:
import sys
import os

# Add the Backend directory dynamically
current_dir = os.getcwd()  # Get the current working directory
sys.path.insert(0, os.path.abspath(os.path.join(current_dir, '../..')))

# Import the required modules
from BusinessLayer.PDFAnalyzer.InformationRetrival   import *


In [2]:


# Example of common words for both languages
common_words_en = {"the", "and", "is", "in", "of", "to","or", "a", "on", "yes", "no", "etc", "it"}
common_words_he = {"של", "הוא", "היא", "זה", "את", "אם","עם", "או","כי"}

# Create an instance of the WordIndex
word_index = WordIndexController(common_words_en, common_words_he)

word_index.hebrew_dict

# pdf1 = "/Users/davidvolodarsky/Downloads/2019-Moed-B.pdf"
pdf_2= "/Users/davidvolodarsky/Downloads/previous exams/2018 Moed B.pdf"
pdf_3= "/Users/davidvolodarsky/Downloads/previous exams/2018 Moed B.pdf"


word_index.process_pdf(pdf_2, "question_dto1")
word_index.process_pdf(pdf_2, "question_dto2")

word_index.english_dict



['question_dto1', 'question_dto2']

In [4]:
# Assuming `word_index.hebrew_dict` is your dictionary
search_word = "תדפיס"

if search_word in word_index.hebrew_dict:
    print(f"Found '{search_word}':", word_index.hebrew_dict[search_word])
else:
    print(f"'{search_word}' not found.")

Found 'תדפיס': ['question_dto1', 'question_dto2']


In [7]:
word_index.hebrew_dict["סיפדת"]


['question_dto1', 'question_dto2']

In [4]:
import pdfplumber
from collections import defaultdict
import re

class WordIndex2:
    def __init__(self, common_words_en, common_words_he):
        self.english_dict = defaultdict(list)
        self.hebrew_dict = defaultdict(list)
        self.common_words_en = set(common_words_en)
        self.common_words_he = set(common_words_he)

    def update_dictionary(self, words, question_dto, dictionary, common_words):
        for word in words:
            # Add components of hyphenated words
            if '-' in word:
                components = word.split('-')
                for component in components:
                    if component not in common_words and question_dto not in dictionary[component]:
                        dictionary[component].append(question_dto)

            # Add the original word (hyphenated or not)
            if word not in common_words and question_dto not in dictionary[word]:
                dictionary[word].append(question_dto)

    def extract_words(self, text):
        if text is None:
            return [], []

        # Regex for English words, including hyphenated ones
        english_words = re.findall(r'\b[a-zA-Z]+(?:-[a-zA-Z]+)?\b', text)

        # Regex for Hebrew words, including hyphenated ones
        hebrew_words = re.findall(r'\b[א-ת]+(?:-[א-ת]+)?\b', text)

        # Additionally, include components of hyphenated words
        split_english = []
        for word in english_words:
            if '-' in word:
                split_english.extend(word.split('-'))
            split_english.append(word)

        split_hebrew = []
        for word in hebrew_words:
            if '-' in word:
                split_hebrew.extend(word.split('-'))
            split_hebrew.append(word)

        return split_english, split_hebrew

    def process_pdf(self, pdf_file_path, question_dto):
        try:
            with pdfplumber.open(pdf_file_path) as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text() + " "  # Combine text from all pages

            # Extract English and Hebrew words
            english_words, hebrew_words = self.extract_words(text)

            # Update dictionaries
            self.update_dictionary(english_words, question_dto, self.english_dict, self.common_words_en)
            self.update_dictionary(hebrew_words, question_dto, self.hebrew_dict, self.common_words_he)

            # Sort dictionaries
            self.english_dict = dict(sorted(self.english_dict.items(), key=lambda x: x[0].lower()))
            self.hebrew_dict = dict(sorted(self.hebrew_dict.items()))
        except Exception as e:
            print(f"Error processing PDF: {e}")

    def search_word(self, word, language="en"):
        if language == "en":
            return self.english_dict.get(word, [])
        elif language == "he":
            return self.hebrew_dict.get(word, [])
        else:
            raise ValueError("Invalid language. Use 'en' for English or 'he' for Hebrew.")


In [6]:
wordIndex2 = WordIndex2(common_words_en, common_words_he)
wordIndex2.process_pdf(pdf_2, "question_dto1")
len(wordIndex2.english_dict)



94

In [9]:
wordIndex2.english_dict["in-order"]

KeyError: 'In-order'